# Electrical Fault Detection and Classification using Line and Phase Currents and Voltages

Electrical power plants require fault detection and operation of protection equipment in minimum possible time to remain stable. The faults on electrical power system transmission lines should be first detected and classified correctly, and repaired in the least possible time. The protection system used for a transmission line can also initiate the other relays to protect the power system from outages. A sound fault detection system provides a fast and secure relaying operation. Once trained on reliable data, an ANN-based fault detection system can help provide fast predictions for the presence of a fault and classify what kind of fault it is.


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [60]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from keras.utils import np_utils
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

# Fault Detection

## Data Preprocessing

In [3]:
!unzip '/content/drive/MyDrive/Datasets/faults.zip' -d '/content'

Archive:  /content/drive/MyDrive/Datasets/faults.zip
  inflating: /content/classData.csv  
  inflating: /content/detect_dataset.csv  


In [4]:
detectionData = pd.read_csv('detect_dataset.csv')

In [5]:
detectionData.head()

,Output (S),Ia,Ib,Ic,Va,Vb,Vc,Unnamed: 7,Unnamed: 8
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431,NaN,NaN
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202,NaN,NaN
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251,NaN,NaN
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963,NaN,NaN
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050,NaN,NaN


In [6]:
detectionData.drop(['Unnamed: 7', 'Unnamed: 8'], axis='columns' ,inplace=True)

In [7]:
detectionData.shape

(12001, 7)

In [8]:
detectionData.head()

,Output (S),Ia,Ib,Ic,Va,Vb,Vc
0,0,-170.472196,9.219613,161.252583,0.054490,-0.659921,0.605431
1,0,-122.235754,6.168667,116.067087,0.102000,-0.628612,0.526202
2,0,-90.161474,3.813632,86.347841,0.141026,-0.605277,0.464251
3,0,-79.904916,2.398803,77.506112,0.156272,-0.602235,0.445963
4,0,-63.885255,0.590667,63.294587,0.180451,-0.591501,0.411050


In [9]:
detectionData['Output (S)'].value_counts()

0    6505
1    5496
Name: Output (S), dtype: int64

In [10]:
Xdetection = detectionData.iloc[:, 1:].values
ydetection = detectionData.iloc[:, 0].values

In [11]:
Xd_train, Xd_test, yd_train, yd_test = train_test_split(Xdetection, ydetection, test_size = 0.2, random_state = 0)

In [12]:
sc = StandardScaler()
Xd_train = sc.fit_transform(Xd_train)
Xd_test = sc.transform(Xd_test)

## ANN Model

In [13]:
detectionANN = Sequential()
detectionANN.add(Dense(units=6, activation='relu'))
detectionANN.add(Dense(units=6, activation='relu'))
detectionANN.add(Dense(units=1, activation='sigmoid'))
detectionANN.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [14]:
detectionANN.fit(Xd_train, yd_train, batch_size = 32, epochs = 100)

Epoch 1/100
300/300 [==============================] - 4s 3ms/step - loss: 0.5607 - accuracy: 0.7136
Epoch 2/100
300/300 [==============================] - 1s 3ms/step - loss: 0.3026 - accuracy: 0.8951
Epoch 3/100
300/300 [==============================] - 1s 3ms/step - loss: 0.1854 - accuracy: 0.9491
Epoch 4/100
300/300 [==============================] - 1s 3ms/step - loss: 0.1154 - accuracy: 0.9697
Epoch 5/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0831 - accuracy: 0.9775
Epoch 6/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0665 - accuracy: 0.9815
Epoch 7/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0574 - accuracy: 0.9837
Epoch 8/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0513 - accuracy: 0.9846
Epoch 9/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0470 - accuracy: 0.9859
Epoch 10/100
300/300 [==============================] - 1s 3ms/step - loss: 0.0441 - accura

## Results

In [15]:
yd_pred = detectionANN.predict(Xd_test)
yd_pred = (yd_pred > 0.5)
print(np.concatenate((yd_pred.reshape(len(yd_pred),1), yd_test.reshape(len(yd_test),1)),1))

[[1 1]
 [0 0]
 [0 0]
 ...
 [0 0]
 [0 0]
 [0 1]]


In [16]:
cm = confusion_matrix(yd_test, yd_pred)
acc = accuracy_score(yd_test, yd_pred)
tn, fp, fn, tp = cm.ravel()
print(f"Correct Predictions : {tn+tp}")
print(f"Wrong Predictions : {fn+fp}")
print(f"Accuracy : {acc*100} %")

Correct Predictions : 2387
Wrong Predictions : 14
Accuracy : 99.41690962099126 %


# Fault Classification

## Data Preprocessing

In [17]:
classData = pd.read_csv('classData.csv')

In [18]:
classData.shape

(7861, 10)

In [72]:
classData.head()

,G,C,B,A,Ia,Ib,Ic,Va,Vb,Vc,fault_types
0,1,0,0,1,-151.291812,-9.677452,85.800162,0.400750,-0.132935,-0.267815,1001
1,1,0,0,1,-336.186183,-76.283262,18.328897,0.312732,-0.123633,-0.189099,1001
2,1,0,0,1,-502.891583,-174.648023,-80.924663,0.265728,-0.114301,-0.151428,1001
3,1,0,0,1,-593.941905,-217.703359,-124.891924,0.235511,-0.104940,-0.130570,1001
4,1,0,0,1,-643.663617,-224.159427,-132.282815,0.209537,-0.095554,-0.113983,1001


In [77]:
Xclass = classData.iloc[:, 4:].values
yclass = classData.iloc[:, 0:4].values

In [21]:
print(Xclass[0,:])

[-1.51291812e+02 -9.67745156e+00  8.58001623e+01  4.00749853e-01
 -1.32934945e-01 -2.67814907e-01]


In [50]:
print(yclass[0,:])

[1 0 0 1]


In [78]:
faults = ["None", "LG Fault", "LL Fault", "LLG Fault", "LLL Fault", "LLLG Fault"]
yc = []
for f in yclass:
  if f[0] == 0 and f[1] == 0 and f[2] == 0 and f[3] == 0:
    yc.append(faults[0])
  elif f[0] == 1 and f[1] == 0 and f[2] == 0 and f[3] == 1:
    yc.append(faults[1])
  elif f[0] == 0 and f[1] == 1 and f[2] == 1 and f[3] == 0:
    yc.append(faults[2])
  elif f[0] == 1 and f[1] == 0 and f[2] == 1 and f[3] == 1:
    yc.append(faults[3])
  elif f[0] == 0 and f[1] == 1 and f[2] == 1 and f[3] == 1:
    yc.append(faults[4])
  else:
    yc.append(faults[5])
yc = np.array(yc)
print(f"No Fault : {len(yc[yc==faults[0]])}")
print(f"LF Fault : {len(yc[yc==faults[1]])}")
print(f"LL Fault : {len(yc[yc==faults[2]])}")
print(f"LLG Fault : {len(yc[yc==faults[3]])}")
print(f"LLL Fault : {len(yc[yc==faults[4]])}")
print(f"LLLG Fault : {len(yc[yc==faults[5]])}")
yclass = yc

No Fault : 2365
LF Fault : 1129
LL Fault : 1004
LLG Fault : 1134
LLL Fault : 1096
LLLG Fault : 1133


In [79]:
encoder = LabelEncoder()
encoder.fit(yclass)
encoded_Y = encoder.transform(yclass)
yclass = np_utils.to_categorical(encoded_Y)

In [80]:
Xc_train, Xc_test, yc_train, yc_test = train_test_split(Xclass, yclass, test_size = 0.2, random_state = 0)

In [81]:
sc = StandardScaler()
Xc_train = sc.fit_transform(Xc_train)
Xc_test = sc.transform(Xc_test)

## ANN Model

In [83]:
classANN = Sequential()
classANN.add(Dense(units=12, activation='relu'))
classANN.add(Dense(units=12, activation='relu'))
classANN.add(Dense(units=6, activation='sigmoid'))
classANN.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [84]:
classANN.fit(Xc_train, yc_train, batch_size = 32, epochs = 100)

Epoch 1/100
197/197 [==============================] - 1s 4ms/step - loss: 1.4433 - accuracy: 0.5060
Epoch 2/100
197/197 [==============================] - 1s 4ms/step - loss: 0.8129 - accuracy: 0.7351
Epoch 3/100
197/197 [==============================] - 1s 4ms/step - loss: 0.4896 - accuracy: 0.8357
Epoch 4/100
197/197 [==============================] - 1s 4ms/step - loss: 0.3413 - accuracy: 0.8930
Epoch 5/100
197/197 [==============================] - 1s 3ms/step - loss: 0.2666 - accuracy: 0.9210
Epoch 6/100
197/197 [==============================] - 1s 3ms/step - loss: 0.2195 - accuracy: 0.9342
Epoch 7/100
197/197 [==============================] - 1s 3ms/step - loss: 0.1888 - accuracy: 0.9413
Epoch 8/100
197/197 [==============================] - 1s 4ms/step - loss: 0.1654 - accuracy: 0.9555
Epoch 9/100
197/197 [==============================] - 1s 4ms/step - loss: 0.1462 - accuracy: 0.9614
Epoch 10/100
197/197 [==============================] - 1s 3ms/step - loss: 0.1307 - accura

## Results

In [95]:
loss, acc = classANN.evaluate(Xc_test, yc_test)
print(f'Accuracy : {acc*100} %')

50/50 [==============================] - 0s 3ms/step - loss: 0.0162 - accuracy: 0.9943
Accuracy : 99.42784309387207 %


# Summary

For Fault Detection, the ANN Model predicted the faults with an accuracy of 99.41% and for Fault Classification, the model was able to classify faults with an accuracy of 99.42%. 